two case studies of extreme precipitation during drought.

 - California Case Study – Event between October 19th -26th 2021
 (forecasts starting on Oct 1, 2021 and go out until Nov 30, 2021)
 
lat_e = 43   <br>
lat_s = 36   <br>
lon_s = -125  <br>
lon_e = -118  <br>

 - Southwest Monsoon Case Study – Event between July 20th – 27th 2021
( use the forecasts starting on Jul 1, 2021 and go out until Aug 31, 2021.)
Drought maps focus on Arizona, but I try to include as much of southwest AZ and New Mexico, therefore approximate boundaries are defined:

lat_e = 37  <br>
lat_s = 31  <br>
lon_s = -115 <br>
lon_e = -102  <br>

1) compute box-average PRISM data for the dates
2) get ERA5 WTs for the dates

3) compute box average ECMWF forecast daily precipitation for the dates (all ensemble members for a single forecast)
4) get ECMWF forecast WTs for the dates (all ensemble members for a single forecast)

In [1]:
import os
from datetime import date, timedelta

import numpy as np
import pandas as pd
import xarray as xr

In [2]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

# write PRISM domain averaged prec. timeseries to a flnm        
def rw_prism_ts(date_s, date_e,lat_s, lat_e, lon_s, lon_e, flnm_o):
    '''
    date_s = date(2021, 10, 1)
    date_e = date(2021, 11, 30)
    lat_e = 43.
    lat_s = 36.
    lon_s = -125.
    lon_e = -118.
    '''
    # create a range of dates list  
    yyyymmdd = [dt.strftime("%Y%m%d") for dt in daterange(date_s, date_e)]
    n_day = len(yyyymmdd)

    # and then convert it to datetime      
    yyyymmdd_datetime = pd.to_datetime(yyyymmdd)
    month_2 =yyyymmdd_datetime.month.values
    day_2  = yyyymmdd_datetime.day.values
    year_2 = yyyymmdd_datetime.year.values
    year_2d = np.expand_dims(year_2, axis=1)
    month_2d = np.expand_dims(month_2, axis=1)
    day_2d = np.expand_dims(day_2, axis=1)
    yyyymmdd_2d = np.concatenate((year_2d, month_2d, day_2d), axis = 1)
    yyyymmdd_2d.shape
    yyyymmdd_datetime_1d = np.expand_dims(yyyymmdd_datetime, axis=1)
    
    #compute box-average PRISM
    dir_i = '/glade/campaign/mmm/c3we/prein/observations/PRISM/data/'
    # The strftime() method returns a string representing date 
    year_s = date_s.strftime("%Y")
    flnm_prism = dir_i + 'PR/PRISM_daily_ppt_'+ str(year_s) + '.nc'
    print(flnm_prism)

    with xr.open_dataset(flnm_prism) as ds_prism:
        prec_1d = ds_prism.PR.sel(time=slice(date_s.strftime("%Y-%m-%d"), date_e.strftime("%Y-%m-%d")), \
                              rlat=slice(lat_e, lat_s), rlon=slice(lon_s, lon_e)) \
                              .mean(dim=('rlat','rlon'))

    # zip aggregates 2 1D lists to a tuple
    prec_tuple = list(zip(yyyymmdd, prec_1d.values.tolist()))

    # Writing a list of tuples to a text file 
    with open(flnm_o, 'w') as file:
        for tuple in prec_tuple:
            file.write('%s  %.2f \n' % tuple)
            
# write ECMWF domain averaged prec. timeseries to a flnm        
def rw_ecmwf_ts(date_s, date_e,lat_s, lat_e, lon_s, lon_e, flnm_o):
    '''
    date_s = date(2021, 10, 1)
    date_e = date(2021, 11, 30)
    lat_e = 43.
    lat_s = 36.
    lon_s = -125.
    lon_e = -118.
    '''
    # create a range of dates list  
    yyyymmdd = [dt.strftime("%Y%m%d") for dt in daterange(date_s, date_e)]
    n_day = len(yyyymmdd)

    # and then convert it to datetime      
    yyyymmdd_datetime = pd.to_datetime(yyyymmdd)
    month_2 =yyyymmdd_datetime.month.values
    day_2  = yyyymmdd_datetime.day.values
    year_2 = yyyymmdd_datetime.year.values
    year_2d = np.expand_dims(year_2, axis=1)
    month_2d = np.expand_dims(month_2, axis=1)
    day_2d = np.expand_dims(day_2, axis=1)
    yyyymmdd_2d = np.concatenate((year_2d, month_2d, day_2d), axis = 1)
    yyyymmdd_2d.shape
    yyyymmdd_datetime_1d = np.expand_dims(yyyymmdd_datetime, axis=1)
    dir_ecmwf = '/glade/campaign/mmm/c3we/ECMWF/'
    yyyymm_s = date_s.strftime("%Y%m")
    flnm_ec = dir_ecmwf + str(yyyymm_s) + '/prec/prec_' + str(yyyymm_s)+ ".nc" 
    print(flnm_ec)

    n_mem = 50
    with xr.open_dataset(flnm_ec) as ds_ec:
        prec_t = ds_ec.TP_GDS0_SFC.sel(g0_lon_3=slice(lon_s, lon_e), g0_lat_2=slice(lat_e, lat_s))
        prec = prec_t[:n_mem, 0:n_day, :,:].mean(dim=('g0_lat_2','g0_lon_3'))
        #n_mem = prec.shape[0]


    # m->mm
    prec_2d = np.asarray(prec.transpose())*1000.
    prec_daily_2d = np.ones((n_day,n_mem)) 

    # calculate daily prec. from Total precipitation
    # because I started from the first one
    prec_daily_2d[0,:] = prec_2d[0,:] 
    for nd in range(1,n_day):
        prec_daily_2d[nd,:] = prec_2d[nd,:] - prec_2d[nd-1,:]  
    
    data_2d = np.concatenate((yyyymmdd_2d, prec_daily_2d), axis = 1)

    fmt = '%i '*3 +' %1.2f'*n_mem
    np.savetxt(flnm_o, data_2d, fmt=fmt)
    
#  get ERA5 WTs from  date_s, date_e
def rw_era5WT_ts(date_s, date_e, flnm_o):
    '''
    date_s = date(2021, 10, 1)
    date_e = date(2021, 11, 30)
    '''
    # create a range of dates list  
    yyyymmdd = [dt.strftime("%Y%m%d") for dt in daterange(date_s, date_e)]
    n_day = len(yyyymmdd)

    # and then convert it to datetime      
    yyyymmdd_datetime = pd.to_datetime(yyyymmdd)
    month_2 =yyyymmdd_datetime.month.values
    day_2  = yyyymmdd_datetime.day.values
    year_2 = yyyymmdd_datetime.year.values
    
    year_s = date_s.strftime("%Y")
    #flnm_wt = '/glade/work/mingge/WTing_existing-centroids/CONUS-WTs_1979-2019_PSL-UV700-TCW_12WTs_ERA5.cla'
    flnm_wt = '/glade/campaign/mmm/c3we/mingge/COEXIST/ERA5/WT/CONUS-WTs_'+ str(year_s) + '.cla'
    print(flnm_wt)

    wts = pd.read_csv(flnm_wt, delimiter = '\t', header=None, index_col=False, names=["year", "month", "day",  "wt"])

    rows = []
    for nd in range(len(day_2)):
        rows.append(wts[(wts['month']==month_2[nd]) & (wts['year']==year_2[nd]) &  (wts['day']==day_2[nd])].values)
     
    # rows is a list of numpy array, convert it to numpy array
    rows_t = np.asarray(rows) #numpy.ndarray (62, 1, 4)
    
    # Writing np.array to a text file 
    with open(flnm_o, 'w') as file:
        for row in rows_t:
            # to avoid numpy array bracket
            aa = ' '.join(map(str, row[0]))
            file.write('%s \n' % (aa))

# get 50-member ECMWF WTs from  date_s, date_e            
def rw_ecmwfWT_ts(date_s, date_e, flnm_o):
    '''
    date_s = date(2021, 10, 1)
    date_e = date(2021, 11, 30)
    '''
    n_mem = 50
    dir_wt = '/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/WT/'
    yyyymm_s = date_s.strftime("%Y%m")

    # and then convert it to datetime  
    yyyymmdd = [dt.strftime("%Y%m%d") for dt in daterange(date_s, date_e)]
    yyyymmdd_datetime = pd.to_datetime(yyyymmdd)
    month_2 =yyyymmdd_datetime.month.values
    day_2  = yyyymmdd_datetime.day.values
    year_2 = yyyymmdd_datetime.year.values
    
    for nm in range(1, n_mem+1):
        flnm_wt = dir_wt + 'CONUS-WTs_' + yyyymm_s + '_' + "{:02d}".format(nm)+ '.cla'
        #print(flnm_wt)
        wts = pd.read_csv(flnm_wt, delimiter = '\t', header=None, index_col=False, names=["year", "month", "day",  "wt"])

        rows = []
        for nd in range(len(day_2)):
            rows.append(wts[(wts['month']==month_2[nd]) & (wts['year']==year_2[nd]) &  (wts['day']==day_2[nd])].values)
     
        # rows is a list of numpy array, convert it to numpy array
        rows_t = np.asarray(rows)
        wt= rows_t[:,:,3] #(61, 1)
     
        if nm == 1:
            print(flnm_wt)
            data_all = np.squeeze(rows_t[:,:,:3]) #(61, 3)
            
        data_all = np.concatenate((data_all, wt), axis = 1) 
        fmt = '%4i'*(3+n_mem)  

    # Writing np.array to a text file 
    np.savetxt(flnm_o, data_all, fmt=fmt)

### 1) compute box-average PRISM and ECMWF(51 members) 

 - California  

In [3]:
date_s = date(2021, 10, 1)
date_e = date(2021, 11, 30)

lat_e = 43.
lat_s = 36.
lon_s = -125.
lon_e = -118.

flnm_o = 'prec_prism_ca.txt'
rw_prism_ts(date_s, date_e,lat_s, lat_e, lon_s, lon_e, flnm_o)

flnm_o = 'prec_ecmwf_ca.txt'
rw_ecmwf_ts(date_s, date_e,lat_s, lat_e, lon_s, lon_e, flnm_o)

flnm_o = 'wt_era5_ca.txt'
rw_era5WT_ts(date_s, date_e, flnm_o)

flnm_o = 'wt_ecmwf_ca.txt'
rw_ecmwfWT_ts(date_s, date_e, flnm_o)


/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_2021.nc
/glade/campaign/mmm/c3we/ECMWF/202110/prec/prec_202110.nc
/glade/campaign/mmm/c3we/mingge/COEXIST/ERA5/WT/CONUS-WTs_2021.cla
/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/WT/CONUS-WTs_202110_01.cla


 - Monsoon

In [4]:
date_s = date(2021, 7, 1)
date_e = date(2021, 8, 31)

lat_e = 37.
lat_s = 31.
lon_s = -115.
lon_e = -102.

flnm_o = 'prec_prism_monsoon.txt'
rw_prism_ts(date_s, date_e,lat_s, lat_e, lon_s, lon_e, flnm_o)

flnm_o = 'prec_ecmwf_monsoon.txt'
rw_ecmwf_ts(date_s, date_e,lat_s, lat_e, lon_s, lon_e, flnm_o)

flnm_o = 'wt_era5_monsoon.txt'
rw_era5WT_ts(date_s, date_e, flnm_o)

flnm_o = 'wt_ecmwf_monsoon.txt'
rw_ecmwfWT_ts(date_s, date_e, flnm_o)

/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_2021.nc
/glade/campaign/mmm/c3we/ECMWF/202107/prec/prec_202107.nc
/glade/campaign/mmm/c3we/mingge/COEXIST/ERA5/WT/CONUS-WTs_2021.cla
/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/WT/CONUS-WTs_202107_01.cla
